In [1]:
from importlib import reload

import datasets
import numpy as np
import torch


import src.models.components.spatiotemporal_encoder

In [4]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [2]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor


feature_extractor = SignLanguageFeatureExtractor()
model = SpatiotemporalEncoder()

src_tensor = np.random.rand(1, 1, 3, 224, 224)
batched_feature = feature_extractor(src_tensor, sampling_rate=25)
out = model(batched_feature)

out.shape

TypeError: Object of type BitImageProcessor is not JSON serializable

In [2]:
from src.models.components.sign_language_net import SignLanguageNet
import torch


model = SignLanguageNet()

src_tensor = torch.rand(1, 1, 3, 224, 224)
out = model(src_tensor)

out.shape

NameError: name 'indices' is not defined